In [58]:
import subprocess
import pickle
import numpy as np
import os
import glob
import cv2
import shutil
import pandas as pd
import os

thumos14_cls_name = ['BK',
                     'BaseballPitch',
                     'BasketballDunk',
                     'Billiards',
                     'CleanAndJerk',
                     'CliffDiving',
                     'CricketBowling',
                     'CricketShot',
                     'Diving',
                     'FrisbeeCatch',
                     'GolfSwing',
                     'HammerThrow',
                     'HighJump',
                     'JavelinThrow',
                     'LongJump',
                     'PoleVault',
                     'Shotput',
                     'SoccerPenalty',
                     'TennisSwing',
                     'ThrowDiscus',
                     'VolleyballSpiking']


tvseries_cls_name = ['BK',
                     'Pick something up',
                     'Point',
                     'Drink',
                     'Stand up',
                     'Run',
                     'Sit down',
                     'Read',
                     'Smoke',
                     'Drive car',
                     'Open door',
                     'Give something',
                     'Use computer',
                     'Write',
                     'Close door',
                     'Go down stairway',
                     'Go up stairway',
                     'Throw something',
                     'Get in/out of car',
                     'Hang up phone',
                     'Eat',
                     'Answer phone',
                     'Clap',
                     'Dress up',
                     'Undress',
                     'Kiss',
                     'Fall/trip',
                     'Wave',
                     'Pour',
                     'Punch',
                     'Fire weapon']


In [59]:
def extract_frames(video_path, dset):
    #video_path : '/home/muser/tech/data_samples/thumos_vid/video_test_0000026'
    # img_path = '/home/muser/techQmark/result_vid/' + video_name + '/imgs'
    img_path = '/opt/ml/demo_video_making/demo_vid/extracted_frames/' + video_name + '/imgs'
    if dset == 'thumos14':
        subprocess.call('ffmpeg -i {} -r 24/1 -qscale:v 2 {}/%06d.png'.format(video_path+'.mp4', img_path), shell=True)
    else:
        subprocess.call('ffmpeg -i {} -r 24/1 -qscale:v 2 {}/%06d.png'.format(video_path+'.mkv', img_path), shell=True)

In [79]:
def put_text_for_obs(image_name, info, dset, resize=False):
    # info : [frame_num, prob, cls, label]
    image = cv2.imread(image_name)
    [height, width] = image.shape[:2]
    font_size = 1
    loc_w, loc_h = 0, 0
    if dset == 'thumos14':
        loc_w, loc_h = 5, 10
        font_size = 0.4
        font_thick = 1
        step = 20
    else:
        loc_w, loc_h = 5, 30
        font_size = 1.
        font_thick = 2
        step = 30
    cv2.putText(image, "Observation", (loc_w, loc_h), cv2.FONT_HERSHEY_DUPLEX, font_size, (255, 0, 0), font_thick)
    cv2.putText(image, "PRED: {} ({:.2f})".format(info[2], info[1]), (loc_w, loc_h+step), cv2.FONT_HERSHEY_DUPLEX, font_size, (255, 0, 0), font_thick)
    cv2.putText(image, "GT: {}".format(info[3]), (loc_w, loc_h + 2*step), cv2.FONT_HERSHEY_DUPLEX, font_size, (0, 0, 255), font_thick)
    cv2.imwrite(image_name, image)

In [78]:
def put_text(image_name, info, dset, resize=False):
    # info : [frame_num, prob, cls, label]
    image = cv2.imread(image_name)
    [height, width] = image.shape[:2]
    font_size = 1
    loc_w, loc_h = 0, 0
    if dset == 'thumos14':
        loc_w, loc_h = 5, 10
        font_size = 0.4
        font_thick = 1
        step = 20
    else:
        loc_w, loc_h = 5, 30
        font_size = 1.
        font_thick = 2
        step = 30
    cv2.putText(image, "Prediction", (loc_w, loc_h), cv2.FONT_HERSHEY_DUPLEX, font_size, (255, 0, 0), font_thick)
    cv2.putText(image, "PRED: {} ({:.2f})".format(info[2], info[1]), (loc_w, loc_h+step), cv2.FONT_HERSHEY_DUPLEX, font_size, (255, 0, 0), font_thick)
    cv2.putText(image, "GT: {}".format(info[3]), (loc_w, loc_h + 2*step), cv2.FONT_HERSHEY_DUPLEX, font_size, (0, 0, 255), font_thick)
    cv2.imwrite(image_name, image)

In [62]:
def record_obs_videos(result_path, video_name, action_cls, obs_start_chunk, obs_end_chunk):
    frm_path = '/opt/ml/demo_video_making/demo_vid/frames_for_obs_demo/' + video_name + '/frms'
    img_path = '/opt/ml/demo_video_making/demo_vid/extracted_frames/' + video_name + '/imgs'

    # copy frames to put text in copied frames
    # subprocess.call('cp {}/* {}/'.format(img_path, frm_path), shell=True)

    nframes = len(glob.glob(frm_path + '/*.png'))
    all_videos_results = pd.read_csv(result_path)
    prop_idx = prop_idx = (all_videos_results['vid name'] == video_name) & (obs_start_chunk <= all_videos_results['end_chunk']) & (all_videos_results['end_chunk'] <= obs_end_chunk)
    this_video_results = all_videos_results[prop_idx][['end_chunk', 'prob', 'pred_cls', 'GT_cls']].values
    for idx, info in enumerate(this_video_results):
        if idx == 0:
            subprocess.call('rm {}/*'.format(frm_path), shell=True)
            for fl in obs_lst:
                subprocess.call('cp {}/{} {}/'.format(img_path, fl, frm_path), shell=True)
        f_start = (int(info[0]) * 32) + 1
        f_end = (int(info[0])+1) * 32
        prob = float(info[1])
        cls = action_cls[int(info[2])]
        label = action_cls[int(info[3])]
        print(f'f_start:{f_start}, f_end:{f_end}')
        for i in range(f_start, f_end+1):
            image_name = frm_path + '/{:06d}.png'.format(i)
            if os.path.exists(image_name):
                put_text_for_obs(image_name, [i, prob, cls, label], dataset)

    # record a video from texted frames
    print('\n\n### Record a video from texted frames ###\n\n')
    subprocess.call(
        'ffmpeg -framerate 24 -start_number {} -i {}/%06d.png -c:v libx264 -profile:v high -level:v 4.0 -crf 18 -pix_fmt yuv420p -frames:v 256 {} '.
        format(int(this_video_results[0][0]* 32)+1, frm_path, '/opt/ml/demo_video_making/demo_vid/result_demo_video/' + video_name+'.mp4'), shell=True) # 24fps짜리 비디오로 만들기

In [67]:
def record_aa_videos(result_path, video_name, action_cls, aa_start_chunk, aa_end_chunk):
    frm_path = '/opt/ml/demo_video_making/demo_vid/frames_for_aa_demo/'+ video_name +'/frms'
    img_path = '/opt/ml/demo_video_making/demo_vid/extracted_frames/' + video_name + '/imgs'
    all_videos_results = pd.read_csv(result_path)
    prop_idx = (all_videos_results['vid name'] == video_name) & (aa_start_chunk <= all_videos_results['a1_end_chunk']) & (all_videos_results['a1_end_chunk'] <= aa_end_chunk)
    this_video_results = all_videos_results[prop_idx][['a1_end_chunk', 'a1_prob', 'a1_pred_cls', 'a1_GT_cls','a2_end_chunk', 'a2_prob', 'a2_pred_cls', 'a2_GT_cls']].values
    for idx, info in enumerate(this_video_results):
        subprocess.call('rm {}/*'.format(frm_path), shell=True)
        for fl in str_lst:
            subprocess.call('cp {}/{} {}/'.format(img_path, fl, frm_path), shell=True)

        a1_start = (int(info[0]) * 32) + 1
        a1_end = (int(info[0])+1) * 32
        a2_start = (int(info[4]) * 32) + 1
        a2_end = (int(info[4])+1) * 32
        a1_prob = float(info[1])
        a2_prob = float(info[5])
        a1_cls = action_cls[int(info[2])]
        a2_cls = action_cls[int(info[6])]
        a1_label = action_cls[int(info[3])]
        a2_label = action_cls[int(info[7])]
        for i in range(a1_start, a1_end+1):
            image_name = frm_path + '/{:06d}.png'.format(i)
            if os.path.exists(image_name):
                put_text(image_name, [i, a1_prob, a1_cls, a1_label],dataset)
            else:
                print('### NO ###')
        for i in range(a2_start, a2_end+1):
            image_name = frm_path + '/{:06d}.png'.format(i)
            if os.path.exists(image_name):
                put_text(image_name, [i, a2_prob, a2_cls, a2_label], dataset)
    
        print('\n\n### Record a video from texted frames ###\n\n')
        subprocess.call(
            'ffmpeg -framerate 24 -start_number {} -i {}/%06d.png -c:v libx264 -profile:v high -level:v 4.0 -crf 18 -pix_fmt yuv420p -frames:v 64 {} '.
            format(a1_start, frm_path, '/opt/ml/demo_video_making/demo_vid/result_demo_video/aa/' + video_name+'_'+str(idx)+'.mp4'), shell=True)

In [91]:
dataset = 'tvseries'
alpha = 0.75
vid_type='video'
obs_start = 804
obs_end = 811
aa_start = 811
aa_end = 813
str_lst = [str(i).zfill(6)+'.png' for i in range(aa_start * 32, (aa_end * 32)+1)]
obs_lst = [str(i).zfill(6)+'.png' for i in range(obs_start * 32, (obs_end * 32)+1)]

if dataset == 'thumos14':
    video_dir = '/opt/ml/demo_video_making/data_samples/thumos/'
    if alpha == 0.75:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha0.75_tsm_thumos14.csv'
    else:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha1.0_tsm_thumos14.csv'
    video_lst = ['video_test_0000026.mp4','video_test_0000505.mp4'] # 
    print('\nMake demo video about {}\n'.format(video_lst[0]))
    act_cls = thumos14_cls_name
else:
    video_dir = '/opt/ml/demo_video_making/data_samples/tvseries/'
    if alpha == 0.75:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha0.75_tsm_tvseries.csv'
    else:
        result_path = '/opt/ml/demo_video_making/demo_vid/result_csv/test_alpha1.0_tsm_tvseries.csv'
    video_lst = ['24_ep4.mkv']
    act_cls = tvseries_cls_name

# for idx, video_name in enumerate(video_lst):
video_name = video_lst[0]
video_name = video_name[:-4] # 확장자 뺀 이름값만 추출
if os.path.isdir('result_demo_video/' + video_name): # result_vid 안에 비디오 이름으로 된 디렉터리 있으면
    shutil.rmtree('result_demo_video/' + video_name, ignore_errors=True) # 싹다제거
print(f'### VIDEO NAME ### : {video_name}')
os.makedirs(os.path.join('/opt/ml/demo_video_making/demo_vid/frames_for_aa_demo',video_name,'frms'), exist_ok=True)
os.makedirs(os.path.join('/opt/ml/demo_video_making/demo_vid/frames_for_obs_demo',video_name,'frms'), exist_ok=True)
os.makedirs(os.path.join('/opt/ml/demo_video_making/demo_vid/extracted_frames',video_name,'imgs'), exist_ok=True)
# os.makedirs(os.path.join('/opt/ml/demo_video_making/demo_vid/frames_for_aa_demo'), exist_ok=True)
print('\n### extract frames from the video... ###\n')
extract_frames(video_dir + video_name, dataset) # video 이름으로 된 폴더 만들고 그안에서 프레임 추출

### VIDEO NAME ### : 24_ep4

### extract frames from the video... ###



ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1602879523915/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, matroska,webm, from '/opt/ml/demo_video_making/data_samples/tvs

In [92]:
if dataset == 'thumos14':
    record_obs_videos(result_path, video_name, thumos14_cls_name, obs_start, obs_end)
else:
    record_obs_videos(result_path, video_name, tvseries_cls_name, obs_start, obs_end)

rm: cannot remove '/opt/ml/demo_video_making/demo_vid/frames_for_obs_demo/24_ep4/frms/*': No such file or directory


f_start:25729, f_end:25760
f_start:25761, f_end:25792
f_start:25793, f_end:25824
f_start:25825, f_end:25856
f_start:25857, f_end:25888
f_start:25889, f_end:25920
f_start:25921, f_end:25952
f_start:25953, f_end:25984


### Record a video from texted frames ###




ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1602879523915/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, image2, from '/opt/ml/demo_video_making/demo_vid/frames_for_obs

In [93]:
if dataset == 'thumos14':
    record_aa_videos(result_path, video_name, thumos14_cls_name, aa_start, aa_end)
else:
    record_aa_videos(result_path, video_name, tvseries_cls_name, aa_start, aa_end)

rm: cannot remove '/opt/ml/demo_video_making/demo_vid/frames_for_aa_demo/24_ep4/frms/*': No such file or directory




### Record a video from texted frames ###




ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1602879523915/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, image2, from '/opt/ml/demo_video_making/demo_vid/frames_for_aa_



### Record a video from texted frames ###




ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1602879523915/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
Input #0, image2, from '/opt/ml/demo_video_making/demo_vid/frames_for_aa_

### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###
### NO ###


### Record a video from texted frames ###




ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7.5.0 (crosstool-NG 1.24.0.131_87df0e6_dirty)
  configuration: --prefix=/opt/conda --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1602879523915/_build_env/bin/x86_64-conda-linux-gnu-cc --disable-doc --disable-openssl --enable-avresample --enable-gnutls --enable-gpl --enable-hardcoded-tables --enable-libfreetype --enable-libopenh264 --enable-libx264 --enable-pic --enable-pthreads --enable-shared --enable-static --enable-version3 --enable-zlib --enable-libmp3lame
  libavutil      56. 51.100 / 56. 51.100
  libavcodec     58. 91.100 / 58. 91.100
  libavformat    58. 45.100 / 58. 45.100
  libavdevice    58. 10.100 / 58. 10.100
  libavfilter     7. 85.100 /  7. 85.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  7.100 /  5.  7.100
  libswresample   3.  7.100 /  3.  7.100
  libpostproc    55.  7.100 / 55.  7.100
[image2 @ 0x5610d8be1180] Could find no file with path '/opt/ml/demo_vide